Francisco Javier Piqueras Martínez

# TP3 - Apartado 2 - Infraestructuras Computacionales para el Procesamiento de Datos Masivos
## Ejercicio 1
Desarrollar un notebook de Jupyter, denominado “tweetCount.ipynb” en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter, se establezca una duración de batch de un segundo, y se muestre, cada 5 segundos, el número de tweets recibidos en los últimos 10 segundos.

En primer lugar, realizamos todos los imports necesarios.

In [1]:
import findspark
findspark.init('/Users/javierpiquerasmartinez/dev/spark-2.4.4-bin-hadoop2.7')

In [2]:
import os
os.environ['JAVA_HOME']='/Library/Java/JavaVirtualMachines/jdk1.8.0_231.jdk/Contents/Home/jre'
os.environ['PYSPARK_SUBMIT_ARGS']='--master local[*] --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

Inicializamos SparkContext y el StreamingContext, con una duración de batch de 1 segundo.

In [4]:
sc = SparkContext(master="local[*]", appName="tweetCount")
ssc = StreamingContext(sc, batchDuration = 1)

ssc.checkpoint("checkpoint")

Para la recepción de datos, creamos un DirectStream que atienda a el evento 'kafkaTwitter' en el servidor localhost:9092.

In [5]:
dstream = KafkaUtils.createDirectStream(
    ssc, 
    topics = ['kafkaTwitter'],
    kafkaParams = {
        'bootstrap.servers' : 'localhost:9092'
    })

Finalmente, mapeamos el número de procesamientos del batch para su devolución:

In [6]:
count_windowed = dstream.countByWindow(10, 5).map(lambda x:('Number of tweets after 5 seconds: %s' % x))

count_windowed.pprint()

In [7]:
ssc.start()

-------------------------------------------
Time: 2020-01-12 21:27:02
-------------------------------------------

-------------------------------------------
Time: 2020-01-12 21:27:07
-------------------------------------------
Number of tweets after 5 seconds: 18

-------------------------------------------
Time: 2020-01-12 21:27:12
-------------------------------------------
Number of tweets after 5 seconds: 115

-------------------------------------------
Time: 2020-01-12 21:27:17
-------------------------------------------
Number of tweets after 5 seconds: 190

-------------------------------------------
Time: 2020-01-12 21:27:22
-------------------------------------------
Number of tweets after 5 seconds: 178

-------------------------------------------
Time: 2020-01-12 21:27:27
-------------------------------------------
Number of tweets after 5 seconds: 188

-------------------------------------------
Time: 2020-01-12 21:27:32
-------------------------------------------
Number 

#### ¿Alrededor de qué número (aproximado) se estabiliza el número de tweets que se procesan en el lapso de tiempo indicado (10 segundos)? ¿Tiene sentido? ¿Por qué?

Las primeras ventanas no tienen registros porque el aún no se ha ejecutado el script, por lo tanto el producer aún no está activo.
La 2º ventana recoge los primeros registros. Sin embargo, tiene pocos: 18.
A partir de la 4º ventana se estabilizan los registros entorno a 185 o 190 registros cada 10 segundos.

Esto ocurre porque hasta que el desplazamiento de la ventana no ha llegado a cubrir en su totalidad el máximo de registros por segundo(tiempo de batch) de procesamiento.